In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [147]:
czantoria_full = pd.read_csv('data/csv/czantoria_sauce.csv')

In [148]:
czantoria_full.columns = ['time', 'distance', 'heartrate', 'cadence', 'velocity_smooth', 'altitude', 'grade_smooth']

In [153]:
czantoria_nonzero = czantoria_full[(czantoria_full['velocity_smooth'] > 0) | (czantoria_full['time'] == 0)]

In [154]:
czantoria_time = czantoria_nonzero.groupby('distance').agg({'time': 'max'}).reset_index()['time']

In [155]:
czantoria_final = pd.merge(czantoria_nonzero, czantoria_time, on=['time'])

In [158]:
czantoria_final

,time,distance,heartrate,cadence,velocity_smooth,altitude,grade_smooth
0,0,0.0,91,0,0.000,414.4,-14.3
1,4,0.4,90,0,0.107,414.4,-14.7
2,5,2.8,90,162,0.560,414.0,-14.7
3,6,5.2,90,162,0.868,413.7,-15.3
4,7,7.8,89,162,1.552,413.3,-14.2
...,...,...,...,...,...,...,...
13380,13875,24137.1,182,158,1.806,846.4,18.2
13381,13876,24137.8,182,158,1.566,846.5,18.0
13382,13877,24138.8,182,158,1.388,846.7,17.0
13383,13879,24140.2,183,162,0.968,846.9,16.4


In [125]:
def get_running_cost(gradient):
    i = gradient
    C_r = 155.4*i**5 - 30.4*i**4 - 43.3*i**3 + 46.3*i**2 + 19.5*i + 3.6
    return C_r

In [126]:
def get_metabolic_energy_expenditure(grade, velocity):
    gradient = grade/100
    C_r = get_running_cost(gradient)
    return C_r * velocity
    